In [1]:
import operator
import unittest

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score

from graphs import datasets
from measure.distance import *
from measure.kernel import *
from measure.kernel_new import *
from measure.shortcuts import *
from scorer import rand_index

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.cluster.hierarchy import fcluster, ward
from scipy.linalg import sqrtm
from ward import Ward
from scipy.cluster._hierarchy import nn_chain

In [2]:
d3_category20 = [
    '#aec7e8',
    '#1f77b4',
    '#ffbb78',
    '#ff7f0e',
    '#98df8a',
    '#2ca02c',
    '#ff9896',
    '#d62728',
    '#c5b0d5',
    '#9467bd',
    '#c49c94',
    '#8c564b',
    '#7f7f7f',
    
    '#aec7e8',
    '#1f77b4',
    '#ffbb78',
    '#ff7f0e',
    '#98df8a',
    '#2ca02c',
    '#ff9896',
    '#d62728',
    '#c5b0d5',
    '#9467bd',
    '#c49c94',
    '#8c564b',
    '#7f7f7f',
] 

In [3]:
def linspace(start, end, count):
    count -= 4
    grid = list(np.linspace(start, end, count))
    step = (end - start) / (count - 1)
    grid.extend([0.1 * step, 0.5 * step, end - 0.1 * step, end - 0.5 * step])
    return grid

In [5]:
kbward_data = {}
graphs, info = datasets.news_2cl_1
for m_num, measure in enumerate(KernelNew.get_all_new()):
    measureparamdict = {}
    for param in tqdm_notebook(linspace(0, 1, 55), desc=measure.__name__):
        mean = []
        for edges, nodes in graphs:
            try:
                D = measure(edges).getK(list(measure(edges).scaler.scale([param]))[0])
                n_clusters=len(list(set(nodes)))
                y_pred = Ward().predict(D, n_clusters)
                ri = adjusted_rand_score(nodes, y_pred)
                mean.append(ri)
            except KeyboardInterrupt:
                raise KeyboardInterrupt()
            except Exception as e:
                print('Error in {} {}\t| {}'.format(measure.__name__, param, e))
            except np.linalg.linalg.LinAlgError as err:
                print('Suka blyat')
        mean = [m for m in mean if m is not None and m == m]
        score = np.array(mean).mean()
        if score is not None and score == score:
            measureparamdict[param] = score
    
    if len(list(measureparamdict.items())) > 0:
        maxparam = max(measureparamdict.items(), key=operator.itemgetter(1))[0]
        print("DONE: {}\tparam: {} score: {}\n\n".format(measure.__name__, maxparam, measureparamdict[maxparam]))
        measureparamdict = np.array(sorted(measureparamdict.items(), key=operator.itemgetter(0)))
        kbward_data[measure(np.zeros((2,2))).name] = measureparamdict
        plt.plot(range(measureparamdict.shape[0]), sorted(measureparamdict[:, 1])[::-1], color=d3_category20[m_num], label=measure(np.zeros((2,2))).name, linewidth=2)
        plt.title(measure.__name__)
        plt.show()
    else:
        print("DONE: {}\tfailed\n\n".format(measure.__name__))

plt.xlim(0, 55)
plt.ylim(.58, .73)
plt.title('python ARI')

A Jupyter Widget

/Users/illusionww/Documents/GitHub/py_graphs/measure/kernel_new.py:45: RuntimeWarning: divide by zero encountered in log
  return np.log(K)


Error in Katz 0.0	| 'NoneType' object has no attribute 'nodes'


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 